EJECUTAR CELDA A CELDA LEYENDO LOS COMENTARIOS, NO ES NECESARIO USAR GPU AUNQUE ES UN POCO MÁS RÁPIDO

En este notebook se cargan los modelos entrenados y se calculan los errores en el conjunto de test.

If on google collab, if not skip the following cell

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#CHANGE ACCORDING TO DIRECTORY
%cd '/content/gdrive/MyDrive/TFG_Matematicas'


Mounted at /content/gdrive
/content/gdrive/MyDrive/TFG_Matematicas


## Preparation to test models

In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dense, Dropout, Input, SimpleRNN, LSTM, Input, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
#import visualkeras
from keras.utils import plot_model
from utils.utils import *


In [ ]:
# UNCOMMENT BELOW IF USING TPU

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# print("All devices: ", tf.config.list_logical_devices('TPU'))
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
# UNCOMMENT BELOW IF USING GPU

# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Load data indexes
train_idxs = read_indexes('./utils/train.txt')
validation_idxs =  read_indexes('./utils/validation.txt')
test_idxs = read_indexes('./utils/test.txt')

In [ ]:
# Load parameters for each set
data_indexes = [(i+1,j+1) for i in range(10) for j in range(2000)]
parameters = load_parameter_list()
df_parameters = pd.DataFrame(parameters, columns=['cx', 'cy', 'a', 'b', 'theta', 'e1', 'e2'])
df_parameters['data_index'] = data_indexes

y_train = df_parameters[df_parameters['data_index'].isin(train_idxs)].drop('data_index', axis = 1)
y_validation = df_parameters[df_parameters['data_index'].isin(validation_idxs)].drop('data_index', axis = 1)
y_test = df_parameters[df_parameters['data_index'].isin(test_idxs)].drop('data_index', axis = 1)

In [ ]:
# Load data (only test needed).
# X_train = np.array(load_data_multithreaded(train_idxs,50))
# print('train loaded')
# X_validation = np.array(load_data_multithreaded(validation_idxs,50))
# print('validation loaded')
X_test = np.array(load_data_multithreaded(test_idxs,50,))
print('test loaded')

test loaded


# Preparing metrics

Prepare custom metrics to monitor parameter e1 and parameter e2

In [ ]:
def e1_mse(y_true, y_pred):
    output = y_pred[:, -2]
    output_mae = tf.keras.metrics.mean_squared_error(y_true[:, -2], output)
    return output_mae
def e2_mse(y_true, y_pred):
    output = y_pred[:, -1]
    output_mae = tf.keras.metrics.mean_squared_error(y_true[:, -1], output)
    return output_mae

def e1_mae(y_true, y_pred):
    output = y_pred[:, -2]
    output_mae = tf.keras.metrics.mean_absolute_error(y_true[:, -2], output)
    return output_mae
def e2_mae(y_true, y_pred):
    output = y_pred[:, -1]
    output_mae = tf.keras.metrics.mean_absolute_error(y_true[:, -1], output)
    return output_mae

def e1_rmse(y_true, y_pred):
    output = y_pred[:, -2]
    output_rmse = rmse(y_true[:, -2], output)
    return output_rmse
def e2_rmse(y_true, y_pred):
    output = y_pred[:, -1]
    output_rmse = rmse(y_true[:, -1], output)
    return output_rmse

In [ ]:
METRICS=[tf.keras.metrics.RootMeanSquaredError(name='rmse'),
         tf.keras.metrics.MeanAbsoluteError(name='mae'),
         e1_mse,e2_mse,e1_mae,e2_mae]

# Directories to load model

In [ ]:
dir_weights = "models/weights" # Directory to store models
dir_training_history = "models/training" # Directory to store models training

# Test samples

The following code generates _size_ random samples. They are used to compare models performance on examples

In [ ]:
test_indices = np.random.choice(len(y_test), size = 3, replace=False)
print(test_indices)
idx_tuples = [test_idxs[i] for i in test_indices]
print(idx_tuples)

[  82 1146  944]
[(1, 1240), (9, 362), (7, 1374)]


The following cells are divided by architecture type. In each section, a code is prepared to test each model on the test set, calculating metrics and examples.

# Feedforward model

In [ ]:
# Model creation
model_name = "feedforward"
feedforward = tf.keras.Sequential([
    Input(shape=(4400, 17)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(7)
])
#Loading tained weights

model_path = dir_weights + f'/best_model_{model_name}.h5'
feedforward.load_weights(model_path)
feedforward.compile(optimizer='adam', loss='mse', metrics=[METRICS])

In [ ]:
# Evaluation
model_evaluation(feedforward, X_test, y_test, test_indices)

45/45 [==============================] - 3s 8ms/step - loss: 0.4825 - rmse: 0.6946 - mae: 0.3815 - e1_mse: 0.5091 - e2_mse: 2.4734 - e1_mae: 0.4463 - e2_mae: 1.1991
Test mse: 0.48248904943466187
Test rmse: 0.6946142911911011
Test mae: 0.3814548850059509
Test R2: 0.5090829730033875
45/45 [==============================] - 0s 6ms/step
Metrics for parameters e1 and e2:
Parameter e1:
  	MSE: 0.518, RMSE: 0.720, MAE: 0.450, R2: -3.571
Parameter e2:
  	MSE: 2.382, RMSE: 1.543, MAE: 1.175, R2: -1.106
1/1 [==============================] - 0s 18ms/step

Random prediction examples
Parameters:	cx	cy	a	b	theta	e1	e2
------------
real:		 -0.056	2.869	1.013	0.572	-0.061	2.732	3.784
prediction:	 -0.194	2.918	0.964	0.596	-0.076	2.658	4.495
------------
real:		 0.230	2.929	1.283	0.218	-0.019	2.482	4.506
prediction:	 0.238	2.864	1.232	0.224	-0.214	2.461	4.159
------------
real:		 0.831	3.210	1.841	0.429	0.291	2.626	3.651
prediction:	 0.662	3.118	1.708	0.481	0.154	2.468	2.792


# Convolutional model

In [ ]:
# Model creation
model_name = "cnn"
cnn = tf.keras.Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(4400, 17, 1), padding='same'),
  BatchNormalization(),
  MaxPooling2D((1, 2)),

  Conv2D(64, (3, 3), activation='relu', padding='same'),
  BatchNormalization(),
  MaxPooling2D((1, 2)),

  Conv2D(64, (3, 3), activation='relu', padding='same'),
  BatchNormalization(),
  MaxPooling2D((1, 2)),

  GlobalMaxPooling2D(),

  Dense(1024, activation='relu'),
  Dense(512, activation='relu'),
  Dense(256, activation='relu'),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dense(7)
])
#Loading tained weights

model_path = dir_weights + f'/best_model_{model_name}.h5'
cnn.load_weights(model_path)
cnn.compile(optimizer='adam', loss='mse', metrics=[METRICS])


In [ ]:
# Evaluation
model_evaluation(cnn, X_test, y_test, test_indices)

45/45 [==============================] - 4s 38ms/step - loss: 0.4407 - rmse: 0.6794 - mae: 0.3857 - e1_mse: 0.6147 - e2_mse: 2.0884 - e1_mae: 0.5259 - e2_mae: 1.0915
Test mse: 0.4406967759132385
Test rmse: 0.6794062852859497
Test mae: 0.3857262432575226
Test R2: 0.6146714687347412
45/45 [==============================] - 2s 32ms/step
Metrics for parameters e1 and e2:
Parameter e1:
  	MSE: 0.624, RMSE: 0.790, MAE: 0.529, R2: -2.793
Parameter e2:
  	MSE: 1.941, RMSE: 1.393, MAE: 1.057, R2: -1.584
1/1 [==============================] - 0s 271ms/step

Random prediction examples
Parameters:	cx	cy	a	b	theta	e1	e2
------------
real:		 -0.056	2.869	1.013	0.572	-0.061	2.732	3.784
prediction:	 0.114	2.916	1.112	0.674	0.127	2.769	4.782
------------
real:		 0.230	2.929	1.283	0.218	-0.019	2.482	4.506
prediction:	 0.346	3.148	1.663	0.312	0.068	2.280	4.551
------------
real:		 0.831	3.210	1.841	0.429	0.291	2.626	3.651
prediction:	 0.797	3.100	1.874	0.601	0.440	2.671	3.465


#  Recurrent Neural Network Model with LSTM

In [ ]:
# Model creation

model_name = "rnn_lstm"
rnn_lstm = tf.keras.Sequential([
    LSTM(25, input_shape=(4400, 17), return_sequences = True),
    Flatten(),
    Dense(1024, activation='leaky_relu'),
    Dense(512, activation='leaky_relu'),
    Dense(256, activation='leaky_relu'),
    Dense(128, activation='leaky_relu'),
    Dense(64, activation='leaky_relu'),
    Dense(7)
])
#Loading tained weights

model_path = dir_weights + f'/best_model_{model_name}.h5'
rnn_lstm.load_weights(model_path)
rnn_lstm.compile(optimizer='adam', loss='mse', metrics=[METRICS])

In [ ]:
# Evaluation
model_evaluation(rnn_lstm, X_test, y_test, test_indices)

45/45 [==============================] - 3s 54ms/step - loss: 0.4400 - rmse: 1.0139 - mae: 0.5301 - e1_mse: 0.5283 - e2_mse: 2.4359 - e1_mae: 0.4459 - e2_mae: 1.1444
Test mse: 0.4400225579738617
Test rmse: 1.0138558149337769
Test mae: 0.5301209092140198
Test R2: 0.528319776058197
45/45 [==============================] - 3s 49ms/step
Metrics for parameters e1 and e2:
Parameter e1:
  	MSE: 0.531, RMSE: 0.728, MAE: 0.444, R2: -3.462
Parameter e2:
  	MSE: 2.228, RMSE: 1.493, MAE: 1.114, R2: -1.251
1/1 [==============================] - 0s 60ms/step

Random prediction examples
Parameters:	cx	cy	a	b	theta	e1	e2
------------
real:		 -0.056	2.869	1.013	0.572	-0.061	2.732	3.784
prediction:	 -0.081	2.837	1.067	0.552	-0.057	2.876	6.059
------------
real:		 0.230	2.929	1.283	0.218	-0.019	2.482	4.506
prediction:	 0.251	2.982	1.202	0.355	-0.088	2.195	5.605
------------
real:		 0.831	3.210	1.841	0.429	0.291	2.626	3.651
prediction:	 0.653	3.088	1.820	0.480	0.201	2.448	3.537
